Importing the required libraries

In [848]:
# !pip install faker

In [849]:
import numpy as np
import pandas as pd
import csv
from faker import Faker
import random
import decimal
import time

In [850]:
fake = Faker()

In [851]:
rand_factor = 1/100       # 1/100 will include a maximum noice of 1000 Mts (basically rand_factor*1L)
timestep = 1
man_steps = 5
m = 20
t = 3600
# n = 3600

# height = int(random.random()*10000)
speed = 70 / 100000
avg_alt = 3000 / 100000 * (0.66+random.random()/3)
alt_var = 1/100000 / 100000
take_off = False
max_dist = 500000 / 100000 
maneuver = 0.01 #To be given a value between 0 and 1 like a boolean and intermediate values will define that much degree of maneuver
fighter = True
maneuver_fact = maneuver * 2 / 1000
maneuver_prob = 0.001 #set this to be the number of times your plane has to maneuver per 1 move setting this to zero will make no maneuver and 1 will make random motion towards the target dest
regular_time = True

In [852]:
src_lat, src_lon  = float(fake.latitude()), float(fake.longitude())
d_lat , d_lon = float(fake.latitude()), float(fake.longitude())
print(src_lat, src_lon,d_lat,d_lon)

-20.997227 -1.081393 38.4010125 -100.295387


In [853]:
d_lat = src_lat + ((np.random.normal(0,1,1)+0.5)*max_dist)
d_lon = src_lon + (np.random.normal(0,1,1)+0.5)*max_dist

d_lat, d_lon = d_lat[0],d_lon[0]
print(d_lat,d_lon)

-23.060505474978882 -6.617093831908477


In [854]:
#Defining the path of latitude

pts_lat=[src_lat]
for i in range(m):
    new_src_lat = src_lat+(d_lat-src_lat+random.random()*rand_factor)/(m-i+1)
    pts_lat.append(new_src_lat)
    src_lat = new_src_lat
pts_lat.append(d_lat)
print(pts_lat)


#Defining the path of longitude

pts_lon=[src_lon]
for i in range(m):
    new_src_lon = src_lon+(d_lon-src_lon+random.random()*rand_factor)/(m-i+1)
    pts_lon.append(new_src_lon)
    src_lon = new_src_lon
pts_lon.append(d_lon)

print(pts_lon)

[-20.997227, -21.095355038996168, -21.193445578405708, -21.29135308823299, -21.38924408430435, -21.48727860019675, -21.585454882609554, -21.683198650590622, -21.78122162584263, -21.879090585019288, -21.977124870776915, -22.07495652768894, -22.172710845443103, -22.270882937241275, -22.368738740482783, -22.466474077503026, -22.563943500681322, -22.662824844020353, -22.760264938129332, -22.860207455704113, -22.95820642813144, -23.060505474978882]
[-1.081393, -1.3449428446345981, -1.6081933469518195, -1.8716169300374847, -2.135060015926916, -2.3981858915838234, -2.6615746620522, -2.9250202653750685, -3.1885832780276457, -3.4521642686428176, -3.7154417944992697, -3.978950370969283, -4.2425350832102495, -4.506297183740659, -4.770028620679384, -5.032802452537329, -5.296005658254396, -5.558390617656582, -5.8216547644054915, -6.084158656439173, -6.345669188683547, -6.617093831908477]


In [855]:
def set_factor(pos,dest_pos,vel,acc,max_vel):
  next_pt = update_pos(pos,vel,acc,0)
  fact = [0,0,0]
  
  target = np.subtract(dest_pos,next_pt)
  # diff = (np.multiply(target,acc)).tolist()
  # diff = [diff[0]>0,diff[1]>0,diff[2]>0]
  dir_target = np.divide(target,np.add(np.absolute(target),0.000000000001))
  dir_vel = np.divide(target,np.add(np.absolute(vel),0.000000000001))
  diff = (dir_target == dir_vel)
  diff = [int(i == False) for i in diff]
  fact = np.multiply(dir_target,np.divide(np.absolute(target),np.add(magnitude(dir_target),0.000000000001))/100)
  # print((np.multiply(fact,diff)))
  # print("pro")
  if(np.linalg.norm(vel+acc)>max_vel):
    # print("parithi")
    # return [0,0,0]
    fact = np.divide(fact,10)
  return (np.multiply(fact,diff)).tolist()

def magnitude(pos):
  return np.linalg.norm(pos)

def dist(pos1,pos2):
  return (np.sqrt(np.sum(np.square(np.subtract(pos1,pos2))))).tolist()

def air_resist(acc):
  acc[2] = acc[2]*5
  return (np.divide(acc,10)).tolist()

def update_pos(pos,vel,acc,noise):
  if(magnitude(acc)>0.00000000000001):
    v = (np.add(vel , acc)).tolist()
    s = (np.divide(np.subtract(np.square(v),np.square(vel)),np.multiply(acc,2))).tolist()
    # print((np.add(np.add(pos,s),np.multiply(np.random.normal(0,1,3),noise))).tolist())
    return (np.add(np.add(pos,s),np.multiply(np.random.normal(0,1,3),noise))).tolist()
  return (np.add(pos,vel)).tolist()

def update_vel(vel,acc,noise):
  return (np.add(np.add(vel,acc),np.random.normal(0,1,3)*noise)).tolist()

def update_acc(pos,dest_pos,vel,max_vel,acc,noise):
  new_acc = np.add(acc,np.random.normal(0,1,3)*noise)
  # print(acc,new_acc)
  new_acc = np.add(new_acc,np.multiply(np.random.rand(3),set_factor(pos,dest_pos,vel,acc,max_vel)))
  # print(set_factor(pos,dest_pos,vel,acc,max_vel))
  new_acc = np.subtract(new_acc,air_resist(new_acc.tolist()))
  # print(new_acc)
  return new_acc.tolist()

def has_crossed(pos,dest_pos,vel,max_vel,acc,noise):
  pres_dir=np.divide(np.subtract(pos,dest_pos),np.absolute(np.subtract(pos,dest_pos)))
  next_pos=update_pos(pos,vel,acc,noise)
  next_dir=np.divide(np.subtract(next_pos,dest_pos),np.absolute(np.subtract(next_pos,dest_pos)))
  return (pres_dir!=next_dir)

  # To be completed before i finish dinner
  pass

def intrapol(init_pos,vel,acc,speed,maneuver_prob,maneuver_fact,noise_fact):
  global timestep, man_steps
  max_vel = speed
  [src_lat,src_lon,src_alt,dest_lat,dest_lon,dest_alt] = init_pos
  dest_pos = [dest_lat,dest_lon,dest_alt]
  [x_vel,y_vel,z_vel] = vel
  [x_acc,y_acc,z_acc] = acc
  lats , lons , alts = [src_lat] , [src_lon] , [src_alt]
  pos = [src_lat,src_lon,src_alt]
  # print(pos)
  man = False
  while(not ((has_crossed(pos,dest_pos,vel,max_vel,acc,noise_fact[0]).any()))):
  # while( not (has_crossed(pos,dest_pos,vel,max_vel,acc,0)).all()):
  # for i in range(100):
    if(man == True):
      acc = np.add(acc , [acc[0]*random.random()/10,acc[1]*random.random()/10,acc[2]*random.random()/10])
      man = False
    else:
      # pos , vel , acc = (update_pos(pos,vel,acc,noise_fact[0])), (update_vel(vel,acc,noise_fact[1]), (update_acc(pos,dest_pos,vel,max_vel,acc,noise_fact[2]))
      pos = update_pos(pos,vel,acc,noise_fact[0])
      vel = update_vel(vel,acc,noise_fact[1])
      acc = update_acc(pos,dest_pos,vel,max_vel,acc,noise_fact[2])
      lats.append(pos[0])
      lons.append(pos[1])
      alts.append(pos[2])
      print (pos)
      man = (random.random() < maneuver_prob)
  return {"pos":[lats, lons, alts],"vel":[x_vel,y_vel,z_vel],"acc":[x_acc,y_acc,z_acc],"last_pt":[lats[-1],lons[-1],alts[-1]]}

# print(update_pos([0,0,0],[4,6.45,4],[12,3,2],2))
# print(update_vel([0,0,0],[12,3,2],2))
# print(update_acc([12,3,2],2))
# print(dist([1,1,1],[0,0,0]))
# print(np.linalg.norm([1,1,1]))276.2601096138686, 0.06902065793247059]
# [77.53461549624465, 276.6945750331236

In [856]:
orig_lat = []
orig_lon = []
orig_alt = []
alt_src = avg_alt + (random.random()-0.5)*2*alt_var
print(pts_lat)
vel = [((pts_lat[1]-pts_lat[0])/((pts_lat[1]-pts_lat[0])**2+(pts_lon[1]-pts_lon[0])**2))*speed,((pts_lon[1]-pts_lon[0])/((pts_lat[1]-pts_lat[0])**2+(pts_lon[1]-pts_lon[0])**2))*speed,0.0001]

if(take_off == False):
    # vel = [random.random()*speed,random.random()*speed,0]
    # vel[1] = np.sqrt(speed - np.square(vel[0]))
    acc = [0,0,0]
    print(np.array(acc).shape)
    for i in range(len(pts_lat)-1):
    # for i in range(2):
        alt_dst = avg_alt - (random.random()-0.5)*2*avg_alt
        pos = [pts_lat[i],pts_lon[i],alt_src,pts_lat[i+1],pts_lon[i+1],alt_dst]
        print("========================"+str((pts_lat[i],pts_lon[i]))+" "+str(alt_src)+"=========================================")
        res = intrapol(pos,vel,acc,speed,maneuver_prob,maneuver_fact,[0,0,0.00005])
        vel = res["vel"]
        acc = res["acc"]
        vect = res["pos"]
        # print(res["pos"])
        alt_src = alt_dst
        orig_lat.extend(vect[0])
        orig_lon.extend(vect[1])
        orig_alt.extend(vect[2])
        [pts_lat[i+1],pts_lon[i+1],alt_src] = res["last_pt"]
        
else:
    for i in range(len(pts_lat)-3):
        pass


# print(np.array([orig_lat,orig_lon,orig_alt]))
print(len(orig_lat))

[-20.997227, -21.095355038996168, -21.193445578405708, -21.29135308823299, -21.38924408430435, -21.48727860019675, -21.585454882609554, -21.683198650590622, -21.78122162584263, -21.879090585019288, -21.977124870776915, -22.07495652768894, -22.172710845443103, -22.270882937241275, -22.368738740482783, -22.466474077503026, -22.563943500681322, -22.662824844020353, -22.760264938129332, -22.860207455704113, -22.95820642813144, -23.060505474978882]
(3,)
========================(-20.997227, -1.081393) 0.022974512873460022=========================================
[-20.998095525520373, -1.0837256642241815, 0.02307451287346002]
[-20.999015365319824, -1.0860759750458238, 0.023181095772282785]
[-21.00006163030512, -1.0884774789584901, 0.02330425522236758]
[-21.001241538750158, -1.090977568106214, 0.023457235185828318]
[-21.002511676178777, -1.0936681249201334, 0.023637427631504872]
[-21.00384352256867, -1.0966582821957445, 0.0238308216824055]
[-21.00522442123914, -1.0999496658892929, 0.0240289865

In [857]:
lts = (avg_alt + np.random.normal(0,0.000001,len(orig_alt)))
orig_alt = np.add(lts,np.divide(orig_alt,10))
try:
  size = orig_alt.shape[1]
except:
  size = len(orig_alt)
orig_alt = orig_alt.reshape(1,size)



In [858]:
# if(type(orig_lon)=="list"):
# size = len(orig_alt)
# else:
#   size = orig_lon.shape[1]
# orig_alt=np.add(orig_alt,np.random.normal(0,0.0001,size))
# print(orig_alt)

start = int(time.time()-(random.random()*10000))
print(start)
# print(orig_alt.shape)
t = start + np.linspace(1,size,size)
t = t.reshape(1,len(t))
orig_alt = np.array(orig_alt)
# orig_alt = orig_alt.reshape(1,len(orig_alt))
print(t.shape)

1596436095
(1, 656)


In [859]:
data = np.array([orig_lat,orig_lon])
data = np.concatenate((t,data),axis=0)
data = np.concatenate((data,orig_alt),axis=0)
data = np.transpose(data)
print(data.shape)
data = np.multiply(data,[1,1,1,100000])
print(data)

(656, 4)
[[ 1.59643610e+09 -2.09972270e+01 -1.08139300e+00  2.52722372e+03]
 [ 1.59643610e+09 -2.09980955e+01 -1.08372566e+00  2.52807733e+03]
 [ 1.59643610e+09 -2.09990154e+01 -1.08607598e+00  2.52924874e+03]
 ...
 [ 1.59643675e+09 -2.30335256e+01 -6.52443002e+00  2.76620324e+03]
 [ 1.59643675e+09 -2.30425341e+01 -6.53584361e+00  2.76845184e+03]
 [ 1.59643675e+09 -2.30519124e+01 -6.54759785e+00  2.77041792e+03]]


In [860]:
d = pd.DataFrame(data=data,columns = ["t","lat","lon","alt"])
d.to_csv('data/traj'+str(count)+'.csv', sep=",", index=False)
count = count +1

In [861]:
print(count)

21


In [862]:
# count = 11

In [865]:
from google.colab import drive 
drive.mount('/mntDrive') 


Drive already mounted at /mntDrive; to attempt to forcibly remount, call drive.mount("/mntDrive", force_remount=True).


In [864]:
! cp "/content/data/traj10.csv" "/mntDrive/My Drive/synthesis/data"